## 5.3 Iterative Verfahren

In [ ]:
import numpy as np

Zunächst implementieren wir die Potenzmethode nach von Mises.

In [ ]:
def potenzmethode_mieses(A, x, n, k=0):
    n1, n2 = A.shape
    n3, = x.shape
    assert (n1==n2), 'Matrix nicht quadratisch'
    assert (n1==n3), 'Matrix und Vektor Dimensionen passen nicht'
    
    lams = []
    for i in range(n):
        x_neu = np.inner(A, x)
        lams.append(x_neu[k] / x[k])
        x = x_neu / np.linalg.norm(x_neu)
    
    return lams

#### Beispiel 5.13 (Potenzmethode nach von Mises)

Wir betrachten die Matrix
$$A = \begin{pmatrix} 2&1&2\\ -1 & 2 & 1 \\ 1 & 2 & 4 \end{pmatrix}$$
und nehmen den Startvektor $x_0 = (1,1,1)^T$. Mit 6 Iterationsschritten bekommen wir dann die Approximation

In [ ]:
A = np.array([[2, 1, 2],
              [-1, 2, 1],
              [1, 2, 4]], dtype=np.double)
x = np.array([1, 1, 1], dtype=np.double)

l = potenzmethode_mieses(A, x, 6, k=0)
print(f'l_{len(l)} = {l[-1]}')

Dies enstpricht einem Fehler von

In [ ]:
print(f'err = {abs(l[-1] - np.linalg.eig(A)[0][2])}')

Um andere Eigenwerte zu bestimmen, implementieren wir nun die inverse Iteration mit Shift nach Wieland. Hierzu verwenden wir unsere Implementierung der LR-Zerlegung mit Pivotisierung.

In [ ]:
from scripts.LR_Zerlegung import LR_zerlegung_mit_pivot, \
    vorwaerts_einsetzen_ohne_diag, rueckwaerts_einsetzen

In [ ]:
def inverse_interation_wieland(A, x, sigma, n, k=0):
    n1, n2 = A.shape
    n3, = x.shape
    assert (n1==n2), 'Matrix nicht quadratisch'
    assert (n1==n3), 'Matrix und Vektor Dimensionen passen nicht'
    
    B = np.array(A - sigma * np.identity(n1), dtype=A.dtype)
    pivot = LR_zerlegung_mit_pivot(B)
    
    lams = []
    for i in range(n):
        xk = x[k]
        
        for p in pivot:
            x[p] = x[[p[1], p[0]]]
        y = vorwaerts_einsetzen_ohne_diag(B, x)
        x_neu = rueckwaerts_einsetzen(B, y)

        mu = x_neu[k] / xk
        lams.append(sigma + 1 / mu)
        x = np.array(x_neu / np.linalg.norm(x_neu, ord=np.inf))
    
    return lams

#### Beispiel 5.15 (Inverse Iteration mit Shift nach Wieland)

Wir betrachten dieselbe Matrix wie zuvor. Nach 4 Schritten der Inversen Iteration, mit Shift $\sigma=2, -1, 4$, erhalten wir jeweils

In [ ]:
A = np.array([[2, -0.1, 0.4], [0.3, -1, 0.4], [0.2, -0.1, 4]])
v = np.array([1.0, 1.0, 1.0])

lam1 = inverse_interation_wieland(A, v, 2, 4, k=0)
lam2 = inverse_interation_wieland(A, v, -1, 4, k=0)
lam3 = inverse_interation_wieland(A, v, 4, 4, k=0)

print(f'lam_1^({len(lam1)}) = {lam1[-1]}')
print(f'lam_2^({len(lam2)}) = {lam2[-1]}')
print(f'lam_3^({len(lam3)}) = {lam3[-1]}')

Daraus ergeben sich die Fehler

In [ ]:
lam = np.linalg.eig(A)[0]
print(f'err(lam_1) = {abs(lam1[-1] - lam[1]):.3e}')
print(f'err(lam_2) = {abs(lam2[-1] - lam[0]):.3e}')
print(f'err(lam_3) = {abs(lam3[-1] - lam[2]):.3e}')